In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from scipy.signal import butter
from scipy import sparse
from scipy import signal
import imageio
import math
from scipy import linalg

In [ ]:
def prpsd2(BVP, FS, LL_PR, UL_PR, BUTTER_ORDER=6, DETREND=False, PlotTF=False, FResBPM = 0.1,RECT=True):
    '''
    Estimates pulse rate from the power spectral density a BVP signal
    
    Inputs
        BVP              : A BVP timeseries. (1d numpy array)
        fs               : The sample rate of the BVP time series (Hz/fps). (int)
        lower_cutoff_bpm : The lower limit for pulse rate (bpm). (int)
        upper_cutoff_bpm : The upper limit for pulse rate (bpm). (int)
        butter_order     : Order of the Butterworth Filter. (int)
        detrend          : Detrend the input signal. (bool)
        FResBPM          : Resolution (bpm) of bins in power spectrum used to determine pulse rate and SNR. (float)
    
    Outputs
        pulse_rate       : The estimated pulse rate in BPM. (float)
    
    Daniel McDuff, Ethan Blackford, January 2019
    Copyright (c)
    Licensed under the MIT License and the RAIL AI License.
    '''

    N = (60*FS)/FResBPM

    # Detrending + nth order butterworth + periodogram
    if DETREND:
        BVP = detrend(np.cumsum(BVP), 100)
    if BUTTER_ORDER:
        [b, a] = signal.butter(BUTTER_ORDER, [LL_PR/60, UL_PR/60], btype='bandpass', fs = FS)
    
    BVP = signal.filtfilt(b, a, np.double(BVP))
    
    # Calculate the PSD and the mask for the desired range
    if RECT:
        F, Pxx = signal.periodogram(x=BVP,  nfft=N, fs=FS, detrend=False);  
    else:
        F, Pxx = signal.periodogram(x=BVP, window=np.hanning(len(BVP)), nfft=N, fs=FS)
    FMask = (F >= (LL_PR/60)) & (F <= (UL_PR/60))
    
    # Calculate predicted pulse rate:
    FRange = F * FMask
    PRange = Pxx * FMask
    MaxInd = np.argmax(PRange)
    pulse_rate_freq = FRange[MaxInd]
    pulse_rate = pulse_rate_freq*60

    # Optionally Plot the PSD and peak frequency
    if PlotTF:
        # Plot PSD (in dB) and peak frequency
        plt.figure()
        plt.plot(F, 10 * np.log10(Pxx))
        plt.plot(pulse_rate_freq, 10 * np.log10(PRange[MaxInd]),'ro')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power (dB)')
        plt.xlim([0, 4.5])
        plt.title('Power Spectrum and Peak Frequency')
            
    return pulse_rate

In [ ]:
def CHROME_DEHAAN(RGB,FS=30):
    LPF = 0.7
    HPF = 2.5
    WinSec = 1.6

    # RGB = process_video(frames)
    FN = RGB.shape[0]
    NyquistF = 1/2*FS
    B, A = signal.butter(3, [LPF/NyquistF, HPF/NyquistF], 'bandpass')

    WinL = math.ceil(WinSec*FS)
    if(WinL % 2):
        WinL = WinL+1
    NWin = math.floor((FN-WinL//2)/(WinL//2))
    S = np.zeros((NWin, 1))
    WinS = 0
    WinM = int(WinS+WinL//2)
    WinE = WinS+WinL
    totallen = (WinL//2)*(NWin+1)
    S = np.zeros(totallen)

    for i in range(NWin):
        RGBBase = np.mean(RGB[WinS:WinE, :], axis=0)
        RGBNorm = np.zeros((WinE-WinS, 3))
        for temp in range(WinS, WinE):
            RGBNorm[temp-WinS] = np.true_divide(RGB[temp], RGBBase)-1
        Xs = np.squeeze(3*RGBNorm[:, 0]-2*RGBNorm[:, 1])
        Ys = np.squeeze(1.5*RGBNorm[:, 0]+RGBNorm[:, 1]-1.5*RGBNorm[:, 2])
        Xf = signal.filtfilt(B, A, Xs, axis=0)
        Yf = signal.filtfilt(B, A, Ys)

        Alpha = np.std(Xf) / np.std(Yf)
        SWin = Xf-Alpha*Yf
        SWin = np.multiply(SWin, signal.windows.hann(WinL))

        if(i == -1):
            S = SWin
        else:
            temp = SWin[:int(WinL//2)]
            S[WinS:WinM] = S[WinS:WinM] + SWin[:int(WinL//2)]
            S[WinM:WinE] = SWin[int(WinL//2):]
        WinS = WinM
        WinM = WinS+WinL//2
        WinE = WinS+WinL
    BVP = S
    return BVP

def POS_WANG(RGB, fs=30):
    WinSec = 1.6
    # RGB = process_video(frames)
    N = RGB.shape[0]
    H = np.zeros((1, N))
    l = math.ceil(WinSec * fs)

    for n in range(N):
        m = n - l
        if m >= 0:
            Cn = np.true_divide(RGB[m:n, :], np.mean(RGB[m:n, :], axis=0))
            Cn = np.mat(Cn).H
            S = np.matmul(np.array([[0, 1, -1], [-2, 1, 1]]), Cn)
            h = S[0, :] + (np.std(S[0, :]) / np.std(S[1, :])) * S[1, :]
            mean_h = np.mean(h)
            for temp in range(h.shape[1]):
                h[0, temp] = h[0, temp] - mean_h
            H[0, m:n] = H[0, m:n] + (h[0])

    BVP = H
    BVP = utils_detrend(np.mat(BVP).H, 100)
    BVP = np.asarray(np.transpose(BVP))[0]
    b, a = signal.butter(1, [0.75 / fs * 2, 3 / fs * 2], btype='bandpass')
    BVP = signal.filtfilt(b, a, BVP.astype(np.double))
    return BVP

def utils_detrend(input_signal, lambda_value):
    signal_length = input_signal.shape[0]
    # observation matrix
    H = np.identity(signal_length)
    ones = np.ones(signal_length)
    minus_twos = -2 * np.ones(signal_length)
    diags_data = np.array([ones, minus_twos, ones])
    diags_index = np.array([0, 1, 2])
    D = sparse.spdiags(diags_data, diags_index,
                (signal_length - 2), signal_length).toarray()
    filtered_signal = np.dot(
        (H - np.linalg.inv(H + (lambda_value ** 2) * np.dot(D.T, D))), input_signal)
    return filtered_signal

def ICA_POH(RGB, FS=30):
    # Cut off frequency.
    LPF = 0.7
    HPF = 2.5
    # RGB = process_video(frames)

    NyquistF = 1 / 2 * FS
    BGRNorm = np.zeros(RGB.shape)
    Lambda = 100
    for c in range(3):
        BGRDetrend = utils_detrend(RGB[:, c], Lambda)
        BGRNorm[:, c] = (BGRDetrend - np.mean(BGRDetrend)) / np.std(BGRDetrend)
    _, S = ica(np.mat(BGRNorm).H, 3)

    # select BVP Source
    MaxPx = np.zeros((1, 3))
    for c in range(3):
        FF = np.fft.fft(S[c, :])
        F = np.arange(0, FF.shape[1]) / FF.shape[1] * FS * 60
        FF = FF[:, 1:]
        FF = FF[0]
        N = FF.shape[0]
        Px = np.abs(FF[:math.floor(N / 2)])
        Px = np.multiply(Px, Px)
        Fx = np.arange(0, N / 2) / (N / 2) * NyquistF
        Px = Px / np.sum(Px, axis=0)
        MaxPx[0, c] = np.max(Px)
    MaxComp = np.argmax(MaxPx)
    BVP_I = S[MaxComp, :]
    B, A = signal.butter(3, [LPF / NyquistF, HPF / NyquistF], 'bandpass')
    BVP_F = signal.filtfilt(B, A, BVP_I.astype(np.double))

    BVP = BVP_F[0]
    return BVP

def ica(X, Nsources, Wprev=0):
    nRows = X.shape[0]
    nCols = X.shape[1]
    if nRows > nCols:
        print(
            "Warning - The number of rows is cannot be greater than the number of columns.")
        print("Please transpose input.")

    if Nsources > min(nRows, nCols):
        Nsources = min(nRows, nCols)
        print(
            'Warning - The number of soures cannot exceed number of observation channels.')
        print('The number of sources will be reduced to the number of observation channels ', Nsources)

    Winv, Zhat = jade(X, Nsources, Wprev)
    W = np.linalg.pinv(Winv)
    return W, Zhat

def jade(X, m, Wprev):
    n = X.shape[0]
    T = X.shape[1]
    nem = m
    seuil = 1 / math.sqrt(T) / 100
    if m < n:
        D, U = np.linalg.eig(np.matmul(X, np.mat(X).H) / T)
        Diag = D
        k = np.argsort(Diag)
        pu = Diag[k]
        ibl = np.sqrt(pu[n - m:n] - np.mean(pu[0:n - m]))
        bl = np.true_divide(np.ones(m, 1), ibl)
        W = np.matmul(np.diag(bl), np.transpose(U[0:n, k[n - m:n]]))
        IW = np.matmul(U[0:n, k[n - m:n]], np.diag(ibl))
    else:
        IW = linalg.sqrtm(np.matmul(X, X.H) / T)
        W = np.linalg.inv(IW)

    Y = np.mat(np.matmul(W, X))
    R = np.matmul(Y, Y.H) / T
    C = np.matmul(Y, Y.T) / T
    Q = np.zeros((m * m * m * m, 1))
    index = 0

    for lx in range(m):
        Y1 = Y[lx, :]
        for kx in range(m):
            Yk1 = np.multiply(Y1, np.conj(Y[kx, :]))
            for jx in range(m):
                Yjk1 = np.multiply(Yk1, np.conj(Y[jx, :]))
                for ix in range(m):
                    Q[index] = np.matmul(Yjk1 / math.sqrt(T), Y[ix, :].T / math.sqrt(
                        T)) - R[ix, jx] * R[lx, kx] - R[ix, kx] * R[lx, jx] - C[ix, lx] * np.conj(C[jx, kx])
                    index += 1
    # Compute and Reshape the significant Eigen
    D, U = np.linalg.eig(Q.reshape(m * m, m * m))
    Diag = abs(D)
    K = np.argsort(Diag)
    la = Diag[K]
    M = np.zeros((m, nem * m), dtype=complex)
    Z = np.zeros(m)
    h = m * m - 1
    for u in range(0, nem * m, m):
        Z = U[:, K[h]].reshape((m, m))
        M[:, u:u + m] = la[h] * Z
        h = h - 1
    # Approximate the Diagonalization of the Eigen Matrices:
    B = np.array([[1, 0, 0], [0, 1, 1], [0, 0 - 1j, 0 + 1j]])
    Bt = np.mat(B).H

    encore = 1
    if Wprev == 0:
        V = np.eye(m).astype(complex)
    else:
        V = np.linalg.inv(Wprev)
    # Main Loop:
    while encore:
        encore = 0
        for p in range(m - 1):
            for q in range(p + 1, m):
                Ip = np.arange(p, nem * m, m)
                Iq = np.arange(q, nem * m, m)
                g = np.mat([M[p, Ip] - M[q, Iq], M[p, Iq], M[q, Ip]])
                temp1 = np.matmul(g, g.H)
                temp2 = np.matmul(B, temp1)
                temp = np.matmul(temp2, Bt)
                D, vcp = np.linalg.eig(np.real(temp))
                K = np.argsort(D)
                la = D[K]
                angles = vcp[:, K[2]]
                if angles[0, 0] < 0:
                    angles = -angles
                c = np.sqrt(0.5 + angles[0, 0] / 2)
                s = 0.5 * (angles[1, 0] - 1j * angles[2, 0]) / c

                if abs(s) > seuil:
                    encore = 1
                    pair = [p, q]
                    G = np.mat([[c, -np.conj(s)], [s, c]])  # Givens Rotation
                    V[:, pair] = np.matmul(V[:, pair], G)
                    M[pair, :] = np.matmul(G.H, M[pair, :])
                    temp1 = c * M[:, Ip] + s * M[:, Iq]
                    temp2 = -np.conj(s) * M[:, Ip] + c * M[:, Iq]
                    temp = np.concatenate((temp1, temp2), axis=1)
                    M[:, Ip] = temp1
                    M[:, Iq] = temp2

    # Whiten the Matrix
    # Estimation of the Mixing Matrix and Signal Separation
    A = np.matmul(IW, V)
    S = np.matmul(np.mat(V).H, Y)
    return A, S

# 330 Window, 300 Stride and no GT offset Correction

In [ ]:
PPG_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/'
PLETH_ROOT = 'dataset_res/'
_trial = 'v_63_2'

green_error_list = []
avg_error_list = []
chrom_error_list = []
pos_error_list = []
ica_error_list = []
for _trial in os.listdir(PPG_ROOT):
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[0:300]
    gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f"GT: {gt_hr}")

    print(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy')
    pleth = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy').mean(1).mean(1)
    green = pleth[:,1]
    green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
    avg = pleth.mean(1)
    avg_hr = prpsd2(avg-np.mean(avg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Avg Est: {avg_hr}    ;   Avg Error: {np.abs(avg_hr - gt_hr)}')
    chrome_dehaan = CHROME_DEHAAN(pleth)
    chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
    pos_wang = POS_WANG(pleth)
    pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
    ica_poh = ICA_POH(pleth)
    ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
    
    green_error_list.append(np.abs(gt_hr - green_hr))
    avg_error_list.append(np.abs(gt_hr - avg_hr))
    chrom_error_list.append(np.abs(gt_hr - chrome_dehaan_hr))
    pos_error_list.append(np.abs(gt_hr - pos_wang_hr))
    ica_error_list.append(np.abs(gt_hr - ica_hr))
    # error = np.abs(green_hr - gt_hr)
    # print(f'Folder: {_trial}')
    # print(f'PID: {"_".join(_trial.split("_")[:-1])}')
    # print(f"Error: {error}")
    # error_list.append(error)
    print('-'*100)
for _trial in os.listdir(PPG_ROOT):
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[300:600]
    gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f"GT: {gt_hr}")

    print(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_300_{_trial}.npy')
    pleth = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_300_{_trial}.npy').mean(1).mean(1)
    green = pleth[:,1]
    green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
    avg = pleth.mean(1)
    avg_hr = prpsd2(avg-np.mean(avg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Avg Est: {avg_hr}    ;   Avg Error: {np.abs(avg_hr - gt_hr)}')
    chrome_dehaan = CHROME_DEHAAN(pleth)
    chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
    pos_wang = POS_WANG(pleth)
    pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
    ica_poh = ICA_POH(pleth)
    ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
    
    green_error_list.append(np.abs(gt_hr - green_hr))
    avg_error_list.append(np.abs(gt_hr - avg_hr))
    chrom_error_list.append(np.abs(gt_hr - chrome_dehaan_hr))
    pos_error_list.append(np.abs(gt_hr - pos_wang_hr))
    ica_error_list.append(np.abs(gt_hr - ica_hr))
    # error = np.abs(green_hr - gt_hr)
    # print(f'Folder: {_trial}')
    # print(f'PID: {"_".join(_trial.split("_")[:-1])}')
    # print(f"Error: {error}")
    # error_list.append(error)
    print('-'*100)
for _trial in os.listdir(PPG_ROOT):
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[600:900]
    gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f"GT: {gt_hr}")

    print(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_600_{_trial}.npy')
    pleth = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_600_{_trial}.npy').mean(1).mean(1)
    green = pleth[:,1]
    green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
    avg = pleth.mean(1)
    avg_hr = prpsd2(avg-np.mean(avg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Avg Est: {avg_hr}    ;   Avg Error: {np.abs(avg_hr - gt_hr)}')
    chrome_dehaan = CHROME_DEHAAN(pleth)
    chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
    pos_wang = POS_WANG(pleth)
    pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
    ica_poh = ICA_POH(pleth)
    ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
    print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
    
    green_error_list.append(np.abs(gt_hr - green_hr))
    avg_error_list.append(np.abs(gt_hr - avg_hr))
    chrom_error_list.append(np.abs(gt_hr - chrome_dehaan_hr))
    pos_error_list.append(np.abs(gt_hr - pos_wang_hr))
    ica_error_list.append(np.abs(gt_hr - ica_hr))
    # error = np.abs(green_hr - gt_hr)
    # print(f'Folder: {_trial}')
    # print(f'PID: {"_".join(_trial.split("_")[:-1])}')
    # print(f"Error: {error}")
    # error_list.append(error)
    print('-'*100)

print("chrom_error_list: ", np.mean(chrom_error_list))
print("pos_error_list: ", np.mean(pos_error_list))
print("ica_error_list: ", np.mean(ica_error_list))
print("green_error_list: ", np.mean(green_error_list))
print("avg_error_list: ", np.mean(avg_error_list))

# 330 Window, 128 Stride and no GT offset Correction

In [ ]:
PPG_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/'
PLETH_ROOT = 'dataset_res/'
_trial = 'v_63_2'

green_error_list = []
avg_error_list = []
chrom_error_list = []
pos_error_list = []
ica_error_list = []
for _trial in os.listdir(PPG_ROOT):
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))

    print(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy')
    pleth_1 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy').mean(1).mean(1)
    pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
    pleth_2 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_300_{_trial}.npy').mean(1).mean(1)
    pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
    pleth_3 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_600_{_trial}.npy').mean(1).mean(1)
    pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
    pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        avg = pleth.mean(1)
        avg_hr = prpsd2(avg-np.mean(avg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Avg Est: {avg_hr}    ;   Avg Error: {np.abs(avg_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
        
        green_error_list.append(np.abs(gt_hr - green_hr))
        avg_error_list.append(np.abs(gt_hr - avg_hr))
        chrom_error_list.append(np.abs(gt_hr - chrome_dehaan_hr))
        pos_error_list.append(np.abs(gt_hr - pos_wang_hr))
        ica_error_list.append(np.abs(gt_hr - ica_hr))
        print('*'*100)
    print('-'*100)

print("chrom_error_list: ", np.mean(chrom_error_list))
print("pos_error_list: ", np.mean(pos_error_list))
print("ica_error_list: ", np.mean(ica_error_list))
print("green_error_list: ", np.mean(green_error_list))
print("avg_error_list: ", np.mean(avg_error_list))

# 330 Window, 128 Stride

In [ ]:
PPG_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/SIGGRAPH_Data/rgb_files/'
PLETH_ROOT = 'dataset_res/'
_trial = 'v_63_2'

green_error_list = []
avg_error_list = []
chrom_error_list = []
pos_error_list = []
ica_error_list = []
for _trial in os.listdir(PPG_ROOT):
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[25:]

    print(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy')
    pleth_1 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_0_{_trial}.npy').mean(1).mean(1)
    pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
    pleth_2 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_300_{_trial}.npy').mean(1).mean(1)
    pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
    pleth_3 = np.load(f'/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/residual_600_{_trial}.npy').mean(1).mean(1)
    pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
    pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        avg = pleth.mean(1)
        avg_hr = prpsd2(avg-np.mean(avg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Avg Est: {avg_hr}    ;   Avg Error: {np.abs(avg_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
        
        green_error_list.append(np.abs(gt_hr - green_hr))
        avg_error_list.append(np.abs(gt_hr - avg_hr))
        chrom_error_list.append(np.abs(gt_hr - chrome_dehaan_hr))
        pos_error_list.append(np.abs(gt_hr - pos_wang_hr))
        ica_error_list.append(np.abs(gt_hr - ica_hr))
        print('*'*100)
    print('-'*100)

In [ ]:
print("chrom_error_list: ", np.mean(chrom_error_list))
print("pos_error_list: ", np.mean(pos_error_list))
print("ica_error_list: ", np.mean(ica_error_list))
print("green_error_list: ", np.mean(green_error_list))
print("avg_error_list: ", np.mean(avg_error_list))